## Tensorflow eager execution implementation

In [38]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print('gpu', gpu)
    tf.config.experimental.set_memory_growth(gpu, True)
    print('memory growth:' , tf.config.experimental.get_memory_growth(gpu))

gpu PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
memory growth: True


In [39]:
import pandas as pd
import numpy as np

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [40]:
mnist_labels = train_data["label"]
mnist_images = train_data.drop(columns=["label"])

In [41]:
x_train = mnist_images.to_numpy().reshape((42000, 28, 28))
y_train = mnist_labels.to_numpy()
x_test = test_data.to_numpy()

x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [42]:
x_train.shape

(42000, 28, 28, 1)

In [43]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test)).batch(32)

In [44]:
train_ds

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [45]:
#model
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(128, activation='relu')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
model = MyModel()

In [46]:
#loss and optimizer

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [47]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [48]:
#define train step and all things registered on gradient tape
def train_step(images, labels):
    with tf.GradientTape() as tape:
        #define our "y"
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients=tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [49]:
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [50]:
#training
EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
#     test_loss.reset_states()
#     test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
#     for test_images, test_labels in test_ds:
#         test_step(test_images, test_labels)    
        
    print(
        f'Epoch {epoch+1},'
        f'Loss {train_loss.result()}, '
        f'Accuracy {train_accuracy.result() * 100}, '
#         f'Test Loss: {test_loss.result()},'
#         f'Test Accuracy: {test_accuracy.result() *100}'
    )
        

Epoch 1,Loss 0.15522687137126923, Accuracy 95.38809967041016, 
Epoch 2,Loss 0.043815549463033676, Accuracy 98.73333740234375, 
Epoch 3,Loss 0.019248588010668755, Accuracy 99.42619323730469, 
Epoch 4,Loss 0.011717048473656178, Accuracy 99.63095092773438, 
Epoch 5,Loss 0.009760420769453049, Accuracy 99.71904754638672, 


In [51]:
x_test.shape

(28000, 784, 1)

In [52]:
preds = model.predict(x_test.reshape((28000, 28, 28, 1)))

In [102]:
solution = [np.argmax(v) for i, v in enumerate(preds)]

In [138]:
df = pd.DataFrame(np_solution,columns=['Label'])


In [136]:
df.to_csv("submission.csv")